# Simplify the language of an article

On serlo.org we have articles that not all students easily understand. Let's have a look at how a large language model (LLM) together with the langchain framework could help us simplify the language a bit!

## Retrieve Serlo article text for testing

We first define some helper function to retrieve a Serlo article text to be used as a test subject:

In [1]:
import json
import requests

def get_article_from_uuid(uuid_article):
    req = requests.post(
        #"https://de.serlo.org/api/frontend/localhost-graphql-fetch",
        "https://api.serlo-staging.dev/graphql",
        headers = {
            "Content-Type": "application/json",
        },
        json = {
            "query": """
            query {
              uuid(id: %d) {
                ... on Article {
                  currentRevision { 
                    id
                    content
                  }
                }
              }
            }
            """ % uuid_article
        }
    )
    
    return req.json()

def get_text_from_object(object):
    strings = []

    if isinstance(object, dict):
        for key, value in object.items():
            if key == 'text':
                # line break may seem nonsensical but we will use it later
                strings.append(value + "\n")
            else:
                strings.extend(get_text_from_object(value))
    elif isinstance(object, list):
        for item in object:
            strings.extend(get_text_from_object(item))

    return strings

Now we can use that to get the text of an article:

In [2]:
uuid_article = 1795
content: str = get_article_from_uuid(uuid_article)['data']['uuid']['currentRevision']['content']
content_object = json.loads(content)
article_text_with_line_breaks = " ".join(get_text_from_object(content_object))
article_text = article_text_with_line_breaks.replace("\n","").replace("  "," ").replace(" ,",",").replace(" .",".").replace("„ ","„").replace(" “","“")
print(article_text_with_line_breaks)

Die 
 Ableitung
  einer Funktion 
 f
  an einer Stelle 
 x
  gibt die 
 Steigung des Graphen der Funktion
  an dieser Stelle an.
 Bezeichnet wird sie zumeist mit 
 f'(x)
 .
 Ist 
 f'(x_0)>0
 , so 
 steigt
  der Graph von 
 f
  an der Stelle 
 x_0
 .
 Ist 
 f'(x_0)<0
 , so 
 fällt
  der Graph von 
 f
  an der Stelle 
 x_0
 .
 An den 
 Extremstellen
  der Funktion und an 
 Terrassenpunkten 
  gilt:
 
 f'(x_0)=0
 .
  
 Die Ableitung spielt daher eine wichtige Rolle bei der 
 Berechnung der Extrema
  und bei der 
 Untersuchung der Monotonie
  einer Funktion.
 Funktionen, die an jeder Stelle 
 x
  der Definitionsmenge eine Ableitung besitzen, nennt man 
 differenzierbar
 . Das Berechnen der Ableitung nennt man 
 Differenzieren
 .
 Für die Berechnung der Ableitung existieren entsprechende 
 Ableitungsregeln
 .
 Definition
 Die Ableitung an einem Punkt
 Die Ableitung ist zunächst nur für einen Punkt 
 \left(x_0\;\left|\;f\left(x_0\right)\right.\right)
   auf dem Graphen einer Funktion 
 f\lef

## Use langchain to change the article text

### OpneAI setup
Use the OpenAI API with GPT-3.5 as the LLM for this first test. We may later try different options like for example Llama-2.

In [3]:
!pip install python-dotenv
!pip install openai

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [4]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

### Langchain setup

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
chat = ChatOpenAI(temperature=0.0)
template_string = """
Ändere den durch drei Backticks abgegrenzten Text im folgenden Stil: {style}. \
Text: ```{text}```
"""
prompt_template = ChatPromptTemplate.from_template(template_string)

Change the values of the variables as you wish:

In [6]:
jahrgangsstufe: int = 8
schulart: str = "Hauptschule"

style = """Deutsch \
in einfacher Sprache, so dass selbst lernschwache %d-Klässler der Schulart %s das Thema gut verstehen
""" % (jahrgangsstufe, schulart)

prompt_to_change_article = prompt_template.format_messages(style=style, text=article_text)
print(prompt_to_change_article[0].content)


Ändere den durch drei Backticks abgegrenzten Text im folgenden Stil: Deutsch in einfacher Sprache, so dass selbst lernschwache 8-Klässler der Schulart Hauptschule das Thema gut verstehen
. Text: ```Die Ableitung einer Funktion f an einer Stelle x gibt die Steigung des Graphen der Funktion an dieser Stelle an. Bezeichnet wird sie zumeist mit f'(x). Ist f'(x_0)>0, so steigt  der Graph von f an der Stelle x_0. Ist f'(x_0)<0, so fällt der Graph von f an der Stelle x_0. An den Extremstellen der Funktion und an Terrassenpunkten  gilt: f'(x_0)=0.   Die Ableitung spielt daher eine wichtige Rolle bei der Berechnung der Extrema und bei der Untersuchung der Monotonie einer Funktion. Funktionen, die an jeder Stelle x der Definitionsmenge eine Ableitung besitzen, nennt man differenzierbar. Das Berechnen der Ableitung nennt man  Differenzieren. Für die Berechnung der Ableitung existieren entsprechende Ableitungsregeln. Definition Die Ableitung an einem Punkt Die Ableitung ist zunächst nur für einen

Now let's see what this generates:

In [7]:
changed_article = chat(prompt_to_change_article)
print(changed_article.content)

Die Ableitung einer Funktion f an einer Stelle x zeigt an, wie steil der Graph der Funktion an dieser Stelle ist. Man nennt sie oft f'(x). Wenn f'(x_0)>0 ist, steigt der Graph von f an der Stelle x_0. Wenn f'(x_0)<0 ist, fällt der Graph von f an der Stelle x_0. An den Extremstellen der Funktion und an Terrassenpunkten gilt: f'(x_0)=0. Die Ableitung ist wichtig, um die Extrema und die Monotonie einer Funktion zu berechnen. Funktionen, bei denen man an jeder Stelle x der Definitionsmenge eine Ableitung berechnen kann, nennt man differenzierbar. Das Berechnen der Ableitung nennt man Differenzieren. Es gibt Regeln, um die Ableitung zu berechnen. Die Ableitung ist zunächst nur für einen Punkt (x_0|f(x_0)) auf dem Graphen einer Funktion f(x) oder für eine Stelle x_0 definiert. Sie wird durch oder alternativ anschaulich erklärt. Man benutzt den Differenzenquotienten, um die Steigung der Sekante durch den Punkt (x_0|f(x_0)) und einen anderen Punkt auf dem Funktionsgraphen zu bestimmen. Die Ide

The changes are subtle but look promising. An issue seems to be removed Latex snippets.

Another thing is that we just destroyed the structure of the JSON, so we cannot display the result directly in the Serlo Editor. An option to try could be to change the prompt template like so:

In [8]:
template_string = """
Ändere in dem Text im durch drei Backticks abgegrenzten JSON im folgenden Stil: {style}. \
Lasse dabei die Struktur des JSON Objekts intakt. 
JSON: ```{content}```
"""

This leads us to the next issue: The prompt message with the JSON with UUID 1795 is 7506 tokens long but the OpenAI API has a limit of 4097 tokens. For demonstration purposes we could use a shorter article 1805 but 1795 is probably not uncommonly long.

Some options to make it work for articles like 1795 are: 
- Split the article if too long.
- Use a LLM or API with longer context. Anthropic's Claude has a context window of 100k tokens.
- Instead of the long JSON Syntax, we could find another way to separate the individual text snippets and replace them in the original JSON.

The last point sounds reasonable, so we change the prompt template like:

In [9]:
template_string = """
Ändere den durch drei Backticks abgegrenzten Text im folgenden Stil: {style}. \
Lasse die Zeilenumbrüche (\n) sowie Leerzeichen am Zeilenanfang hierbei intakt. \
Text: ```
{text}
```
"""
prompt_template = ChatPromptTemplate.from_template(template_string)

prompt_to_change_article = prompt_template.format_messages(style=style, text=article_text_with_line_breaks)
print(prompt_to_change_article[0].content)


Ändere den durch drei Backticks abgegrenzten Text im folgenden Stil: Deutsch in einfacher Sprache, so dass selbst lernschwache 8-Klässler der Schulart Hauptschule das Thema gut verstehen
. Lasse die Zeilenumbrüche (
) sowie Leerzeichen am Zeilenanfang hierbei intakt. Text: ```
Die 
 Ableitung
  einer Funktion 
 f
  an einer Stelle 
 x
  gibt die 
 Steigung des Graphen der Funktion
  an dieser Stelle an.
 Bezeichnet wird sie zumeist mit 
 f'(x)
 .
 Ist 
 f'(x_0)>0
 , so 
 steigt
  der Graph von 
 f
  an der Stelle 
 x_0
 .
 Ist 
 f'(x_0)<0
 , so 
 fällt
  der Graph von 
 f
  an der Stelle 
 x_0
 .
 An den 
 Extremstellen
  der Funktion und an 
 Terrassenpunkten 
  gilt:
 
 f'(x_0)=0
 .
  
 Die Ableitung spielt daher eine wichtige Rolle bei der 
 Berechnung der Extrema
  und bei der 
 Untersuchung der Monotonie
  einer Funktion.
 Funktionen, die an jeder Stelle 
 x
  der Definitionsmenge eine Ableitung besitzen, nennt man 
 differenzierbar
 . Das Berechnen der Ableitung nennt man 
 Diff

And see how this works:

In [10]:
changed_article_with_line_breaks = chat(prompt_to_change_article).content
print(changed_article_with_line_breaks)

Die Ableitung einer Funktion an einer Stelle x gibt die Steigung des Graphen der Funktion an dieser Stelle an. Bezeichnet wird sie zumeist mit f'(x).

Ist f'(x_0)>0, so steigt der Graph von f an der Stelle x_0.

Ist f'(x_0)<0, so fällt der Graph von f an der Stelle x_0.

An den Extremstellen der Funktion und an Terrassenpunkten gilt:

f'(x_0)=0.

Die Ableitung spielt daher eine wichtige Rolle bei der Berechnung der Extrema und bei der Untersuchung der Monotonie einer Funktion.

Funktionen, die an jeder Stelle x der Definitionsmenge eine Ableitung besitzen, nennt man differenzierbar. Das Berechnen der Ableitung nennt man Differenzieren.

Für die Berechnung der Ableitung existieren entsprechende Ableitungsregeln.

Definition
Die Ableitung an einem Punkt
Die Ableitung ist zunächst nur für einen Punkt (x_0|f(x_0)) auf dem Graphen einer Funktion f(x) bzw. für eine Stelle x_0 definiert. Sie ist gegeben durch

oder alternativ

Anschaulich erhält man durch den Differenzenquotienten die Steigun

Well... that doesn't look right with the newline characters. Though below we see that it might just be what the notebook displays as newline. 

A problem seems to be that the changes to simplify the language are not made like previously. The first sentences read like the original text.

Anyways, let's get it back to JSON:

In [11]:
def get_new_text_into_object(object, text_snippets, list_index=0): 
    if isinstance(object, dict):
        for key, value in object.items():
            if key == 'text':
                value = text_snippets[list_index]
                list_index += 1
            else:
                get_new_text_into_object(value, text_snippets, list_index=list_index)
    elif isinstance(object, list):
        for item in object:
            get_new_text_into_object(item, text_snippets, list_index=list_index)

In [12]:
text_snippets = changed_article_with_line_breaks.split("\n")
get_new_text_into_object(content_object, text_snippets)
print(content_object)

{'plugin': 'article', 'state': {'introduction': {'plugin': 'articleIntroduction', 'state': {'explanation': {'plugin': 'text', 'state': [{'type': 'p', 'children': [{'text': 'Die '}, {'text': 'Ableitung', 'strong': True}, {'text': ' einer Funktion '}, {'type': 'math', 'src': 'f', 'inline': True, 'children': [{'text': 'f'}]}, {'text': ' an einer Stelle '}, {'type': 'math', 'src': 'x', 'inline': True, 'children': [{'text': 'x'}]}, {'text': ' gibt die '}, {'text': 'Steigung des Graphen der Funktion', 'strong': True}, {'text': ' an dieser Stelle an.'}]}, {'type': 'p', 'children': [{'text': 'Bezeichnet wird sie zumeist mit '}, {'type': 'math', 'src': "f'(x)", 'inline': True, 'children': [{'text': "f'(x)"}]}, {'text': '.'}]}]}, 'multimedia': {'plugin': 'image', 'state': {'src': 'https://assets.serlo.org/6227206de64da_69fce42ecb2dedad112be172f4e98b90d71456b5.svg', 'caption': {'plugin': 'text', 'state': [{'type': 'p', 'children': [{}]}]}}}, 'illustrating': True, 'width': 50}}, 'content': {'plugi

We can try to paste that into the Serlo editor in staging to get an idea of what it did.

Conclusion so far: LLMs can help to simplify the language of pure text. Our articles are not pure text, though, but instead given as JSON objects. It would be great to have an automatized way to end up with a JSON, otherwise we can only display a text suggestion to authors that they then have to put themselves into the editor between the other content like images or Latex formulas.